### Example of DNNs in regression - Housing dataset

In [1]:
print("hello")

hello


Loading data

In [2]:
from tensorflow.keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

print(train_data.shape)
print(test_data.shape)
print(max(train_targets), min(train_targets))

(404, 13)
(102, 13)
50.0 5.0


In [10]:
train_data

array([[-0.27224633, -0.48361547, -0.43576161, ...,  1.14850044,
         0.44807713,  0.8252202 ],
       [-0.40342651,  2.99178419, -1.33391162, ..., -1.71818909,
         0.43190599, -1.32920239],
       [ 0.1249402 , -0.48361547,  1.0283258 , ...,  0.78447637,
         0.22061726, -1.30850006],
       ...,
       [-0.40202987,  0.99079651, -0.7415148 , ..., -0.71712291,
         0.07943894, -0.67776904],
       [-0.17292018, -0.48361547,  1.24588095, ..., -1.71818909,
        -0.98764362,  0.42083466],
       [-0.40422614,  2.04394792, -1.20161456, ..., -1.30866202,
         0.23317118, -1.15392266]])

Standardizing data

In [3]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

Building the model (deine function to enable more flexibility in tests)

In [4]:
from tensorflow.keras import models
from tensorflow.keras import layers

def build_model(hidden = 64):
    model = models.Sequential()
    model.add(layers.Dense(hidden, activation='relu',
                            input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(hidden, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [5]:
model = build_model()
model.fit(train_data, train_targets, epochs=80, batch_size=16, verbose=1)

test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)
print(test_mse_score, test_mae_score)

Epoch 1/80
26/26 [==============================] - 1s 2ms/step - loss: 518.8594 - mae: 20.8765
Epoch 2/80
26/26 [==============================] - 0s 2ms/step - loss: 360.8512 - mae: 16.8117
Epoch 3/80
26/26 [==============================] - 0s 2ms/step - loss: 193.2533 - mae: 11.2255
Epoch 4/80
26/26 [==============================] - 0s 2ms/step - loss: 96.3541 - mae: 7.4145
Epoch 5/80
26/26 [==============================] - 0s 2ms/step - loss: 55.4223 - mae: 5.6407
Epoch 6/80
26/26 [==============================] - 0s 2ms/step - loss: 37.4604 - mae: 4.5117
Epoch 7/80
26/26 [==============================] - 0s 2ms/step - loss: 28.8991 - mae: 3.8527
Epoch 8/80
26/26 [==============================] - 0s 2ms/step - loss: 24.2660 - mae: 3.4550
Epoch 9/80
26/26 [==============================] - 0s 2ms/step - loss: 21.3069 - mae: 3.2174
Epoch 10/80
26/26 [==============================] - 0s 2ms/step - loss: 19.1170 - mae: 3.0792
Epoch 11/80
26/26 [==============================] - 

In [6]:
import numpy as np
k = 5
num_val_samples = len(train_data) // k 
num_epochs = 100
all_scores = []
model = build_model()
for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i*num_val_samples: (i+1)*num_val_samples]
    val_targets = train_targets[i*num_val_samples: (i+1)*num_val_samples]
    partial_train_data = np.concatenate( [train_data[:i*num_val_samples], train_data[(i+1)*num_val_samples:]], axis=0)
    partial_train_targets = np.concatenate( [train_targets[:i*num_val_samples], train_targets[(i+1)*num_val_samples:]], axis=0)
    model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)
print(all_scores)
print(np.mean(all_scores))

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
[1.853084921836853, 1.615808129310608, 1.5772215127944946, 1.8083841800689697, 1.715542197227478]
1.7140081882476808
